<a href="https://colab.research.google.com/github/derek-shing/ProofOfConcept/blob/master/Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Testing the Speech to Text Cantonese API


1.   Preprocessing the audio file


*   How to record sound
*   turn to correct format
*   display codec spec.
*   Save to Storage



2.   Using AutoML API
*    Read from Storage
*    Call the API
*    Store the result to Firestore

https://cloud.google.com/firestore/docs/quickstart-servers


https://cloud.google.com/speech-to-text/docs/quickstart  

Convert mp3 to wav:

https://www.online-convert.com/result/b85182ca-bb4a-47a1-9d58-86e22f150a87







In [0]:
project_id = 'my-project-1558599394309'

In [0]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!gcloud config set project {project_id}

Updated property [core/project].


In [9]:
!gsutil cp gs://food1430/newSong.wav newSong.wav

Copying gs://food1430/newSong.wav...
- [1 files][ 10.1 MiB/ 10.1 MiB]                                                
Operation completed over 1 objects/10.1 MiB.                                     


In [11]:
#Check information of a wav file

!ffprobe -i /content/newSong.wav -show_streams -select_streams a:0

ffprobe version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2007-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --en

In [0]:
!ffmpeg -i cantonese_flac.flac -ac 1 mono.flac

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
!wget https://www.signalogic.com/melp/eng_noise/orig/car.wav

--2020-01-24 08:03:49--  https://www.signalogic.com/melp/eng_noise/orig/car.wav
Resolving www.signalogic.com (www.signalogic.com)... 209.150.126.178
Connecting to www.signalogic.com (www.signalogic.com)|209.150.126.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256044 (250K) [audio/x-wav]
Saving to: ‘car.wav’

car.wav             100%[===================>] 250.04K  --.-KB/s    in 0.06s   

2020-01-24 08:03:49 (4.41 MB/s) - ‘car.wav’ saved [256044/256044]



In [0]:
!gsutil cp /content/car.wav gs://food1430/car.wav

Copying file:///content/car.wav [Content-Type=audio/x-wav]...
-
Operation completed over 1 objects/250.0 KiB.                                    


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/My Drive/GCP_account.json"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
pip install google-cloud-speech

     |████████████████████████████████| 92kB 2.8MB/s 


In [10]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
#file_name = os.path.join(
#    os.path.dirname(__file__),
#    'resources',
#    'audio.raw')

file_name='/content/newSong.wav'

# Loads the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    enable_speaker_diarization=True,
    diarization_speaker_count=2,
    language_code='yue-Hant-HK')

# Detects speech in the audio file
response = client.recognize(config, audio)

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

ValueError: ignored

In [0]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums


def sample_long_running_recognize(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100

    # The language of the supplied audio
    language_code = "yue-Hant-HK"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
    return response

In [0]:
uri="gs://food1430/food01-07-20_wav.wav"
data = sample_long_running_recognize(uri)

Waiting for operation to complete...
Transcript: 美式無限無限寬頻人人有份好歡迎大家收聽今日嘅無限寬頻我已經為大家可以畫3d今日呢我就準備咗一間餐廳同你講下因為我覺得你會鍾意食嘅嘢嚟嘅你每次都話我鍾意食嘅嘢好容易搵嘅啫因為我可以食嘅咁有啲嘢係你冇咁鍾意食㗎嘛甜品呢樣嘢呢其實係日本日本日本餐廳啦咁後日本餐都好多種㗎嘛食好多嘢嘅我覺得係比較
Transcript: 被人哋啲就識玩譬如係一個唔係亞洲人啦可能係其他文化嘅人都會接受食呢樣嘢因為唔係嗰啲server迪生嘢食食完返去你會驚我哋飲住先我哋約下中國飲茶食嗰啲鳳爪佢哋又會驚㗎咁樣嘢呢就基本上佢哋唔會驚嘅食緊嚟㗎咁所以係比較大就快啲囉應該就唔會拉麵咁普通啦唔係會唔會係日本嘅定食好似metallica嗰啲嘢嘅舖頭舖頭別在咩呢佢開鋪頭嗰個人下低嗰個主廚個師傅呢佢並唔係日本人哋嘅咁個故事我得以下其實佢個鼻老人
Transcript: 秘魯人呢就走咗去日本嗰度我諗佢嗰度住呀發展呀咁樣呀今日著咗好多年嘅時間佢就係出去餐廳返工呀去咗一間專門頂tempura嘅地方咁跟住呢基本上就學先返嚟呀即係其實佢本身呢好鍾意食呢一樣嘢愛上咗對於呢個文化對於有嘢食咁跟住呢佢就決定咗要將呢樣嘢帶返返嚟洛杉磯咁就喺度開間餐廳都係交畀阿神嘅餐廳嚟嘅下午一啲都唔貴唔會有好靚嘅裝修但係呢個嘢食出殯呢就非常之好食嘅而且呢喺短短的開咗唔夠一年嘅時間先嚟啦已經得到好好嘅平台以咁我真係好有興趣喎因為我自己本身呢係鍾意食tempura嘅但我而家講緊唔係話即係隨便咋嘢嗰啲呀係真係專業整tempura
Transcript: 鋪頭好多嘅餐廳都會有tempura買咁但係嗰啲emperor如果你話畀喺專業專門做tempura個舖頭呢係一個好大嘅差距假我而家諗緊其實我以前呢就有間好出名呀仔畀我嗰邊呀我哋講真係好貴嘅其實有一間喺toris嗰邊都開咗好多年㗎喇咁我以前都耐唔耐都會去食嘅但係你依家呢我相信我就係唔知道嘅所以我好期待你嘅介紹呀好貴呀我都食過啦但係都未食完呢間之後呢你就發覺原來只係十零蚊都唔使十蚊你就可以食到算係真係唔錯呀得兩張相差唔多嘅呢個地方呀你又係一樣唔係咁容易嘅唔係話我求其返屋企埋一堆雜粉麵粉雞蛋返嚟呢就可以做到㗎因為我哋自己屋企第一少咋嘢啦第二其實嗰個
Transcript: 難㗎所有嘢要搬入得非常之好無論係用咩油啦

In [0]:
data

In [0]:
data.results[0].alternatives[0].transcript

'美式無限無限寬頻人人有份好歡迎大家收聽今日嘅無限寬頻我已經為大家可以畫3d今日呢我就準備咗一間餐廳同你講下因為我覺得你會鍾意食嘅嘢嚟嘅佢每次都話我鍾意食嘅嘢好容易搵嘅啫因為我可以食嘅咁有啲嘢係你冇咁鍾意食㗎嘛甜品呢樣嘢呢其實就先講星期日本日本日本餐廳啦咁後日本餐都好多種㗎嘛食好多嘢嘅我覺得係比較'

In [0]:
s="\344\272\272\345\223\213\345\225\262\350\255\254\345\246\202\344\277\202\344\270\200\345\200\213\345\224\224\344\277\202\344\272\236\346\264\262\344\272\272\345\225\246\345\217\257\350\203\275\344\277\202\345\205\266\344\273\226\346\226\207\345\214\226\345\230\205\344\272\272\345\221\242\351\203\275\346\234\203\346\216\245\345\217\227\351\243\237\345\221\242\346\250\243\345\230\242\345\233\240\347\202\272\345\224\224\344\277\202\345\227\260\345\225\262\347\224\237\345\230\242\351\243\237\351\243\237\345\256\214\350\277\224\345\216\273\344\275\240\346\234\203\351\251\232\346\210\221\345\223\213\351\243\262\344\275\217\345\205\210\346\210\221\345\223\213\347\264\204\344\270\213\344\270\255\345\234\213\351\243\262\350\214\266\351\243\237\345\227\260\345\225\262\351\263\263\347\210\252\344\275\242\345\223\213\345\217\210\346\234\203\351\251\232\343\227\216\345\222\201\346\250\243\345\230\242\345\221\242\345\260\261\345\237\272\346\234\254\344\270\212\344\275\242\345\223\213\345\224\224\346\234\203\351\251\232\345\230\205\351\243\237\347\267\212\345\232\237\343\227\216\345\222\201\346\211\200\344\273\245\344\277\202\346\257\224\350\274\203\345\244\247\345\260\261\345\277\253\345\225\262\345\233\211\346\207\211\350\251\262\345\260\261\345\224\224\346\234\203\346\213\211\351\272\265\345\222\201\346\231\256\351\200\232\345\225\246\345\224\224\344\277\202\346\234\203\345\224\224\346\234\203\344\277\202\346\227\245\346\234\254\345\230\205\345\256\232\351\243\237\345\227\260\345\225\262\345\230\242\345\230\205\350\210\226\351\240\255\347\211\271\345\210\245\345\234\250\345\222\251\345\221\242\344\275\242\351\226\213\351\213\252\351\240\255\345\227\260\345\200\213\344\272\272\344\270\213\344\275\216\345\227\260\345\200\213\344\270\273\345\273\232\345\200\213\345\270\253\345\202\205\345\221\242\344\275\242\344\270\246\345\224\224\344\277\202\346\227\245\346\234\254\344\272\272\345\223\213\345\230\205\345\222\201\345\200\213\346\225\205\344\272\213\345\245\275\345\276\227\346\204\217\344\270\213\345\205\266\345\257\246\344\275\242\345\200\213\351\274\273\350\200\201\344\272\272"

In [0]:
s

In [0]:
type(data.results[0].alternatives[0].transcript)

str

#Putting the result to Firestore

In [5]:
pip install --upgrade google-cloud-firestore

     |████████████████████████████████| 337kB 2.5MB/s 


In [0]:
from google.cloud import firestore

# Project ID is determined by the GCLOUD_PROJECT environment variable
db = firestore.Client()

In [0]:
import json

In [0]:
count=1
d={}
for result in data.results:
  key='transcript'+str(count)
  d[key]=result.alternatives[0].transcript
  count+=1



In [0]:
len(d)

19

In [0]:
d

{'transcript1': '美式無限無限寬頻人人有份好歡迎大家收聽今日嘅無限寬頻我已經為大家可以畫3d今日呢我就準備咗一間餐廳同你講下因為我覺得你會鍾意食嘅嘢嚟嘅你每次都話我鍾意食嘅嘢好容易搵嘅啫因為我可以食嘅咁有啲嘢係你冇咁鍾意食㗎嘛甜品呢樣嘢呢其實係日本日本日本餐廳啦咁後日本餐都好多種㗎嘛食好多嘢嘅我覺得係比較',
 'transcript10': '比華利山嗰啲就係食咗麥卡錫啦然後去師傅就好似好高級嘅嗰啲方式食啦你而家講緊呢啲就係基本上一個lunch呀或者係平時搞解決咗個肚餓問題嘅一種選擇嚟㗎好平民化價錢相當之豐到咁而且你啱啱所講嘅因為人哋專門做嘅咁我相信佢嘅品質出品呢係絕對係即係專門店舖頭嘅掂啦其實每個人都有自己嘅有自己嘅喜好啦就算真係鍾意食tempura嘅人有啲蝦嘅有啲人唔鍾意食其他嘅其實你啱啱所講嘅炸饅魚呢係我鍾意食嘅男仔入嚟㗎但我唔係嗰啲㗎我係食隻唔係有執嗰啲鰻魚呀我係鍾意食嗰啲淡水鰻魚真係冇嗰啲日子都要吊銷執嗰啲鰻魚所以兩種食法係唔同嘅我鍾意食嗰種就係淡水嘅冇汁嘅蛋',
 'transcript11': '哪個方式去做過嘅呢係我其中一個最鍾意食嘅tempura點隻品種之一嚟㗎有埋嗰個站鰻魚嘅我哋都係呀同埋一堆嗰啲菜啦你知我勁怕聲都唔出嚟呢係冇聲嚟㗎即係佢係叫你啦非常之好咁夜瞓呢可能係有唔同嘅境界就食個飯嘅食唔到啲嘢其實好啱㗎嘛係會食到嗰個比例啱啱好啦咁你都係食到好嘅食材啦當然你唔可以要求佢畀最靚嘅咩哈你食啦呢啲咁但係我覺得都係覺得佢啦',
 'transcript12': '唔記得當年阿姐而家講緊快十年前嘅事啦你因為佢接飛機就錯過咗一次息tempura嘅機會啦你記唔記得就係一啲好高級嘅我macarthur嘅方式啦咁其實你啱啱講到喇因為出面有好多嘅舖頭咁無論如何去啲lunch嘅時間呢佢都會有一啲所謂嘅我食呀夾tempura令到大部分嘅民眾去到日本餐廳食tempura呢都有一個固有嘅stella態度有個錯覺就差啲嘢出嚟唔係咁重要嘅啫但係當你食咗呢一啲真正嘅member之後呢你就知道你其實對tempura嘅理解呢係一個好錯誤因為如果咋得靚嘅係真係好好食啱啱畫3d所講嘅嗰個脆漿啦個咋分呀如果人哋係真係黃金色嘅咋到兼職薄薄一層',
 'transcript13': '出面嘅你食落去呢真係嗰種趣同埋成口粉呢係完全兩回事嚟㗎一種就係食真正tempura同埋出

In [0]:
doc_ref = db.collection(u'food').document(u'010720')
doc_ref.set(d)

update_time {
  seconds: 1580638803
  nanos: 373966000
}

秘魯人呢就走咗去日本嗰度我諗佢嗰度住呀發展呀咁樣呀今日著咗好多年嘅時間佢就係出去餐廳返工呀去咗一間專門頂tempura嘅地方咁跟住呢基本上就學先返嚟呀即係其實佢本身呢好鍾意食呢一樣嘢愛上咗對於呢個文化對於有嘢食咁跟住呢佢就決定咗要將呢樣嘢帶返返嚟洛杉磯咁就喺度開間餐廳都係交畀阿神嘅餐廳嚟嘅下午一啲都唔貴唔會有好靚嘅裝修但係呢個嘢食出殯呢就非常之好食嘅而且呢喺短短的開咗唔夠一年嘅時間先嚟啦已經得到好好嘅平台以咁我真係好有興趣喎因為我自己本身呢係鍾意食tempura嘅但我而家講緊唔係話即係隨便咋嘢嗰啲呀係真係專業整tempura',

鋪頭好多嘅餐廳都會有tempura買咁但係嗰啲emperor如果你話畀喺專業專門做tempura個舖頭呢係一個好大嘅差距假我而家諗緊其實我以前呢就有間好出名呀仔畀我嗰邊呀我哋講真係好貴嘅其實有一間喺toris嗰邊都開咗好多年㗎喇咁我以前都耐唔耐都會去食嘅但係你依家呢我相信我就係唔知道嘅所以我好期待你嘅介紹呀好貴呀我都食過啦但係都未食完呢間之後呢你就發覺原來只係十零蚊都唔使十蚊你就可以食到算係真係唔錯呀得兩張相差唔多嘅呢個地方呀你又係一樣唔係咁容易嘅唔係話我求其返屋企埋一堆雜粉麵粉雞蛋返嚟呢就可以做到㗎因為我哋自己屋企第一少咋嘢啦第二其實嗰個',

難㗎所有嘢要搬入得非常之好無論係用咩油啦有嘅溫度啦你教唔同嘅嘢需要嘅時間都係唔同嘅咁當然重點點樣去瞓啦咁所以我覺得其實如果你要學識我咁都係幾難嘅嘢囉我覺得係唔容易㗎係其實我以前呢就學個job to run下係跟嗰個日本嘅師傅呀咁所以其實佢有教過我一啲q門喇淨係做呢個分張呢都係用好多嘅步驟同埋公聚假並唔係我哋所講嘅有啲麵粉呀有雞蛋呀點樣呀撈埋一齊呀點樣啦冇撚喺水嘅溫度啦呢啲粉嘅粗細喇你要低過佢啦然後個比例啦等等啦全部都有一個好講究嘅工序同埋份量㗎咁所以點解有啲tempura舖頭有啲人覺得就成間舖頭定係食炸template',

需唔需要做到咁專業呀需唔需要做咁簡單呀咁樣其實呢如果你試過呢啲真係專門食平啲嘅舖頭呢你好難去返去其他鋪頭食tempura㗎啦咁去到咁遠呀我就真係暈暈咁多其實我覺得你好想食嗰啲你啲call呢一間食炸豬扒好食嘅係諗咗好耐真係好慘呀食一餐我都決定咗去食西餐廳因為隻豬扒有冇食過呀嘛咁佢一陣間呢個時候呢就係中午嚟㗎咁我中午嘅其實因為係生活都幾多人㗎所以唔使等位大都係即可換曬嘅咁其實好簡單好似你話齋佢根本呢就專門做嗰幾樣嘢好想食嗰樣因為我去打畀我吖我到咗先下晝一點',

就係用呢一個蟹肉呀patrick整嘅成本價錢唔貴喎佢呢個都係咁嘅做法呢就有兩種選擇一呢就係做嗰種嗰啲當即係上面係食物啦下面就煩得好好因為呢都好食嘅一碗白飯就分開跟住有冇麵豉湯跟住呢就係分開嘅我就自己個人呢就比較呢啲呢啲環境我覺得好撚煩囉因為你會將佢炸物香味啦同埋佢有個特世界上第一混合埋一齊好食嘅重點係咩你所有埋佢嘅飯呢佢都有一隻流心蛋',



流心蛋呢係講明㗎啦佢好佢唔會唔會話畀人聽留心中間嗰個彈琴嘅唔會㗎佢教你將個打咩等個蛋糕嘅樓落個飯度連埋佢本身混合咗嗰度比較特別嘅自由呢一齊食真係好好呀非常之好啦佢唔會過甜會過鹹個味道啱啱好嘅就算你load咗煩到你唔會覺得嘅時候攞返啲全部都係打得好好玩啲嘢就有咩都有炸蝦喇有大雨啦有炸紫菜同埋有打過一陣時見到kappa嘅咁如果你畀貴少少呢就會有一個係炸鰻魚嘅係唔好啦咁我本身自己唔鍾意食鰻魚咁就冇20蚊如果我好似平時賣',

而家問一問其實一啲都唔貴囉我已經流曬好衰但係真係唔係咁容易食到呀全羅南基就真係得嗰十隻手指數唔夠嘅餐廳啦咁你呢間講嘅呢你就而家我聽你咁都目前為止嘅講法呢我就已經係好想嘅自動ii佢嘅好處呢我覺得係因為唔貴呀你講緊有十二蚊到最貴都係去到20蚊咁定係應該可能貴啲但係你講咁嘅價錢呢有咩食到唔錯佢嗰陣都嚟呢已經係非常之難得㗎啦咁你就比華利山嗰啲食緊講冇辦法定200蚊一位㗎嘛咁平民化啲嘢我覺得你係非常之好食㗎啦'}

# Yelp API


*   Search
*   KeyWord



In [0]:
import requests
import json

endpoint="https://api.yelp.com/v3/businesses/search"


key = db.collection('APIKey').document('yelp').get().to_dict()
header = {'Authorization': 'bearer %s' %key['API_Key']}

para={
    'term':'tempura',
    'location':'torrance',
    'limit':10
}

In [0]:
response = requests.get(url=endpoint,params=para,headers=header)

In [0]:
businesses = response.json()['businesses']

In [19]:
len(businesses)

10

In [24]:
for biz in businesses:
  print(biz['name'])

Tendon Tempura Carlos Junior
Matsui
Kagura
Komatsu
Inaba Restaurant
Otafuku Restaurant
Ichimi-an
Ginza AYCE Sushi - Torrance
Koshiji
Wadatsumi
